<a href="https://colab.research.google.com/github/Mahdi-Miri/Signal_Proccesing/blob/main/Signal_Proccesing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Data Readig

In [2]:
!pip install obspy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 98.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 73.1 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.43
    Uninstalling SQLAlchemy-2.0.43:
      Successfully uninstalled SQLAlchemy-2.0.43
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.54 which is incompatible.
google-adk 1.15.1 requires sqlalchemy<3.0.0,>=2.0, but you have sqlalchemy 1.4.54 which is incompatible.


In [9]:
!pip install obspy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 2.5 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.43
    Uninstalling SQLAlchemy-2.0.43:
      Successfully uninstalled SQLAlchemy-2.0.43
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.54 which is incompatible.
google-adk 1.15.1 requires sqlalchemy<3.0.0,>=2.0, but you have sqlalchemy 1.4.54 which is incompatible.


In [1]:
from obspy.clients.fdsn import Client
import glob
from scipy.signal import iirnotch, filtfilt
import os
import numpy as np
import matplotlib.pyplot as plt
from obspy.core import UTCDateTime, read, Stream
from obspy.core.inventory.inventory import read_inventory
from obspy.clients.fdsn import Client
client = Client("IRIS")  # Initialize Client to download data from IRIS
import glob
from phasecorr2.tfpws_stack import tfpws_stack_STFT_band_9375
from scipy.signal import iirnotch, filtfilt
from phasecorr2.phasecorr_seismic import acorr
import os
import numpy as np
import matplotlib.pyplot as plt
from obspy.core import UTCDateTime, read, Stream
from obspy.core.inventory.inventory import read_inventory


ModuleNotFoundError: No module named 'phasecorr2'

In [2]:
def plot_spectrum(data, sampling_rate, title):
    n = len(data)
    fft_data = np.fft.rfft(data)
    freqs = np.fft.rfftfreq(n, 1 / sampling_rate)
    plt.figure(figsize=(8, 4))
    plt.plot(freqs, np.abs(fft_data))
    plt.title(title)
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Amplitude")
    plt.grid(True)
    plt.show()

In [3]:
def date_maker(year, month, day,hour, minute, second):
    year = str(int(year))
    month = str(int(month))
    day = str(int(day))
    hour = str(int(hour))
    minute = str(int(minute))
    second = str(int(second))

    if len(month) == 1:
        month = '0' + month
    if len(day) == 1:
        day = '0' + day
    if len(hour) == 1:
        hour = '0' + hour
    if len(minute) == 1:
        minute = '0' + minute
    if len(second) == 1:
        second = '0' + second

    YMDHMS = year + "-" + month + "-" + day + "-" + hour + "-" + minute + "-" + second
    return YMDHMS


In [4]:
client = Client("IRIS")
# Define the seismic network code
network = "GH"
# Define the list of stations to process
# station_list = ['AKOS', 'KLEF', 'KUKU', 'MRON', 'SHAI', 'WEIJ']
station_list = ['AKOS'] # For this run, only process 'AKOS' station
# Create a formatted date-time string using the custom function for the start date
YMDHMS = date_maker(2013, 1, 1, 0, 0, 0.0)
# Define the main directory for waveform data
Main_direc = "./Waveforms/"

In [ ]:
import obspy

# Read the seismic waveform data from a local MiniSEED file
st = read("/content/drive/MyDrive/Signal/2013-01-01-00-00-00.mseed")
# Plot the raw waveform data
st.plot()
# Define the start time as an ObsPy UTCDateTime object
start_time = UTCDateTime(2013, 1, 1, 0, 0, 0.0)
# Define the directory containing station metadata XML files
xml_dir = "/content/drive/MyDrive/Signal"
# Find all XML files in the directory
xml_files = glob.glob(os.path.join(xml_dir, "*.xml"))
# Read and merge all XML files into a single inventory object
inventory = None
for xml_file in xml_files:
    inv = read_inventory(xml_file)
    if inventory is None:
        inventory = inv
    else:
        inventory += inv

##Modelling

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Input, Conv2D, BatchNormalization, ReLU, MaxPooling2D,
    GlobalAveragePooling2D, TimeDistributed, Dense, Permute, Reshape, Layer
)
from tensorflow.keras.models import Model
import numpy as np

In [ ]:
# --- Custom Layer Definition with Functional Implementation ---
# This layer now has a functional implementation based on the
# Auto-Correlation mechanism using Fast Fourier Transform (FFT).
class CustomAutocorrelationLayer(Layer):
    """
    A custom layer that calculates the temporal auto-correlation of an input tensor.
    It operates on the last axis, which is assumed to be the time dimension.
    The calculation is performed efficiently in the frequency domain using FFT.
    """
    def __init__(self, **kwargs):
        super(CustomAutocorrelationLayer, self).__init__(**kwargs)

    def call(self, inputs):
        # The input shape is expected to be (Batch, Height, Width, TimeSteps)
        # We need to compute auto-correlation along the TimeSteps axis (axis=-1).

        # Get the length of the time series
        sequence_length = tf.shape(inputs)[-1]

        # --- Step 1: Go to Frequency Domain using FFT ---
        # Perform Fast Fourier Transform. tf.signal.rfft is used for real-valued inputs.
        # The length of the FFT is padded to the next power of 2 for efficiency,
        # but for simplicity, we'll use the original length here.
        fft_result = tf.signal.rfft(inputs, fft_length=[sequence_length])

        # --- Step 2: Compute Power Spectral Density ---
        # This is where the correlation is calculated.
        # It's done by multiplying the FFT result by its complex conjugate.
        # This is equivalent to convolution in the time domain.
        power_spectral_density = fft_result * tf.math.conj(fft_result)

        # --- Step 3: Go back to Time Domain using Inverse FFT ---
        # Perform Inverse Real Fast Fourier Transform to get the auto-correlation series.
        autocorr_result = tf.signal.irfft(power_spectral_density, fft_length=[sequence_length])

        return autocorr_result

    def get_config(self):
        # Required for model saving and loading
        config = super(CustomAutocorrelationLayer, self).get_config()
        return config

# --- Full Architecture Definition ---
def build_spatio_temporal_model(input_shape=(60, 60, 100)):
    """
    Builds the complete Spatio-Temporal feature extraction model based on the diagram.

    Args:
        input_shape (tuple): The shape of the input data (Height, Width, TimeSteps).

    Returns:
        keras.Model: The compiled Keras model.
    """
    # Define the input layer for our spatio-temporal data
    inputs = Input(shape=input_shape)

    # --- Part 1: Temporal Transformation & Reshaping ---

    # Apply the functional autocorrelation layer to find temporal patterns.
    # This layer processes each spatial point's time series (60x60 series of length 100).
    # Output values now represent the strength of temporal correlations.
    temporal_features = CustomAutocorrelationLayer()(inputs)  # Shape: (None, 60, 60, 100)

    # Permute the dimensions to bring the time-steps to the front for the next stage.
    # (Height, Width, TimeSteps) -> (TimeSteps, Height, Width)
    permuted = Permute((3, 1, 2))(temporal_features)  # Shape: (None, 100, 60, 60)

    # Reshape to add a 'channels' dimension. Each time-step is now a 60x60x1 image,
    # ready to be processed by a 2D CNN.
    reshaped = Reshape((100, 60, 60, 1))(permuted)  # Shape: (None, 100, 60, 60, 1)

    # --- Part 2: Time-Distributed Spatial Feature Extraction ---

    # Define the CNN block that extracts spatial features from a single 60x60 frame.
    cnn_block = tf.keras.Sequential([
        # Finds low-level features like edges and gradients.
        Conv2D(32, kernel_size=(3, 3), padding='same'),
        BatchNormalization(),
        ReLU(),
        # Downsamples the feature map to make representations more robust.
        MaxPooling2D(pool_size=(2, 2)),

        # Finds higher-level features by combining low-level ones.
        Conv2D(64, kernel_size=(3, 3), padding='same'),
        BatchNormalization(),
        ReLU(),
        MaxPooling2D(pool_size=(2, 2)),

        # Summarizes all spatial features in the frame into a single, fixed-size vector.
        GlobalAveragePooling2D()
    ], name='internal_cnn_block')

    # Apply the defined cnn_block to each of the 100 time-steps independently.
    # The output is a sequence of 100 feature vectors, one for each time-step.
    time_distributed_cnn = TimeDistributed(cnn_block)(reshaped)  # Shape: (None, 100, 64)

    # --- Part 3: Final Transformation ---

    # A Dense layer refines the features from the CNN, reducing each feature vector's size to 10.
    # It learns combinations of the spatial features.
    dense_features = Dense(10, activation='relu')(time_distributed_cnn)  # Shape: (None, 100, 10)

    # Final permutation to get the output shape of (Features, TimeSteps).
    # This format might be useful for downstream tasks that analyze each feature over time.
    final_output = Permute((2, 1))(dense_features)  # Shape: (None, 10, 100)

    # Create the final model by defining its inputs and outputs
    model = Model(inputs=inputs, outputs=final_output, name='SpatioTemporal_Feature_Extractor')

    return model

# --- Model Creation, Compilation, and Execution Example ---

if __name__ == '__main__':
    # Define the input shape as per the diagram
    INPUT_SHAPE = (60, 60, 100)

    # Build the model
    model = build_spatio_temporal_model(input_shape=INPUT_SHAPE)

    # Compile the model. A loss function and optimizer are needed for training.
    # For feature extraction, you might not train it, but compilation is a good practice.
    model.compile(optimizer='adam', loss='mse')

    # Print the model summary to verify the architecture and shapes
    print("--- Model Summary ---")
    model.summary()

    # --- Example of running the model with dummy data ---
    print("\n--- Running a test prediction ---")
    # Create a batch of 2 dummy "videos" with random data
    dummy_data = np.random.rand(2, *INPUT_SHAPE)

    # Get the model's prediction
    predictions = model.predict(dummy_data)

    # Print the shape of the output to confirm it matches the design
    print(f"Input data shape: {dummy_data.shape}")
    print(f"Final output shape: {predictions.shape}")


--- Model Summary ---


Model: "SpatioTemporal_Feature_Extractor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 60, 60, 100)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ custom_autocorrelation_layer    │ (None, None, None,     │             0 │
│ (CustomAutocorrelationLayer)    │ 100)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ permute (Permute)               │ (None, 100, None,      │             0 │
│                                 │ None)                  │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 100, 60, 60, 1) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 100, 64)        │        19,200 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 100, 10)        │           650 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ permute_1 (Permute)             │ (None, 10, 100)        │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,850 (77.54 KB)

 Trainable params: 19,658 (76.79 KB)

 Non-trainable params: 192 (768.00 B)


--- Running a test prediction ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step
Input data shape: (2, 60, 60, 100)
Final output shape: (2, 10, 100)
